## Introduction to Machine Learning  

## Assignment 2: Decision Trees

You can't learn technical subjects without hands-on practice. The assignments are an important part of the course. To submit this assignment you will need to make sure that you save your Jupyter notebook. 

Below are the links of 2 videos that explain:

1. [How to save your Jupyter notebook](https://youtu.be/0aoLgBoAUSA) and,       
2. [How to answer a question in a Jupyter notebook assignment](https://youtu.be/7j0WKhI3W4s).

### Assignment Learning Goals:

By the end of the module, students are expected to:

- Broadly describe how decision trees make predictions.
- Use `DecisionTreeClassifier()` and `DecisionTreeRegressor()` to build decision trees using scikit-learn.
- Use the `.fit()` and `.predict()` paradigm and use `.score()` method of ML models.
- Explain the concept of decision boundaries.
- Build a decision tree classifier on a real-world dataset and explore different hyperparameters of the classifier.
- Explain how decision boundaries change with `max_depth`.
- Build a decision tree regressor.

This assignment covers [Module 2](https://ml-learn.mds.ubc.ca/en/module2) of the online course. You should complete this module before attempting this assignment.

Any place you see `...`, you must fill in the function, variable, or data to complete the code. Substitute the `None` with your completed code and answers then proceed to run the cell!

Note that some of the questions in this assignment will have hidden tests. This means that no feedback will be given as to the correctness of your solution. It will be left up to you to decide if your answer is sufficiently correct. These questions are worth 2 points.

In [139]:
# Import libraries needed for this lab
from hashlib import sha1

import altair as alt
import graphviz
import numpy as np
import pandas as pd

from IPython.display import HTML
from sklearn import tree
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.tree import DecisionTreeClassifier
import test_assignment2 as t

from display_tree import display_tree
alt.renderers.enable('html')

RendererRegistry.enable('html')

## 1. Decision Tree Structure 

**Question 1.1** <br> {points: 5}  

<img src='static/tree.png' width="70%"/>

Label the 4 components of the decision tree diagram each with one of the possible values: 

- Stump

- Root

- Branch

- Trunk

- Node 

- Leaf

- Bark

- Nodule

*Answer in the cell below by assigning the name of the decision tree as a string to the objects named `label_1`, `label_2`, `label_3` and `label_4`.*

In [140]:
label_1 = 'Root'
label_2 = 'Branch'
label_3 = 'Leaf'
label_4 = 'Node'

In [141]:
t.test_1_1_1(label_1)

'Success'

In [142]:
t.test_1_1_2(label_2)

'Success'

In [143]:
# check that the variable exists
assert 'label_3' in globals(
), "Please make sure that your solution is named 'label_3'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

In [144]:
t.test_1_1_4(label_4)

'Success'

**Question 1.2** <br> {points: 1}  


<img src='static/pman_tree.png' width="130%"/>

What would this decision tree predict for an observation with the following features?

<br>

```
   attack  defense  sp_attack  sp_defense  speed  capture_rt  gen
0    33      101        52         23        74      12        5
```
<br>

*Save you answer as a string in an object named `pokemon_prediction`.*

In [145]:
def get_pokemon_class(stats) :
    '''     
    Returns the class of a pokemon based on its stats.        

    Parameters
    ----------
    stats : dict
        The dict with pokemon stats
    
    Returns
    -------
    str
        The class of the pokemon
    
    Examples
    --------
    >>> get_pokemon_class(stats)
    'Reg'
    '''

    if stats['capture_rt'] <= 20.0 :
        pokemon_class = 'Reg'
        if stats['sp_attack'] <= 63.5 :
            pokemon_class = 'Lengend'
            if stats['sp_defense'] <= 90.0 :
                pokemon_class = 'Reg' # Leaf 1
            else :
                pokemon_class = 'Lengend' # Leaf 2
        elif stats['defense'] <= 135.0 :
            pokemon_class = 'Lengend' # Leaf 3
        elif stats['speed'] <= 100.0 :
            pokemon_class = 'Lengend' # Leaf 4
        else :
            pokemon_class = 'Reg' # Leaf 5

    elif stats['attack'] <= 175.0 :
        pokemon_class = 'Reg'

        if stats['sp_attack'] <= 112.5 :
            pokemon_class = 'Reg'
            if stats['sp_defense'] <= 99.5 :
                pokemon_class = 'Reg' # Leaf 6
            else :
                pokemon_class = 'Reg' # Leaf 7
        elif stats['gen'] <= 5.5 :
            pokemon_class = 'Reg' # Leaf 8
        else :
            pokemon_class = 'Lengend' # Leaf 9
    else :
        pokemon_class = 'Lengend' # Leaf 10

    return pokemon_class

In [146]:
pokemon_stats = {
    'attack' : 33,
    'defense' : 101,
    'sp_attack' : 52,
    'sp_defense' : 23,
    'speed' : 74,
    'capture_rt' : 12,
    'gen' : 5
}

display(pokemon_stats)

{'attack': 33,
 'defense': 101,
 'sp_attack': 52,
 'sp_defense': 23,
 'speed': 74,
 'capture_rt': 12,
 'gen': 5}

In [147]:
pokemon_prediction = get_pokemon_class(pokemon_stats)

display(pokemon_prediction)

'Reg'

In [148]:
t.test_1_2(pokemon_prediction)

'Success'

**Question 1.3** <br> {points: 1}  

What is the depth of the decision tree in **Question 1.2**?



*Answer in the cell below with  your answer and assign it to an object called `tree_depth`.*

In [149]:
tree_depth = 4

In [150]:
t.test_1_3(tree_depth)

'Success'

## 2. Decision Tree Building

Suppose you have three different job offers with comparable salaries and job descriptions. You want to decide which one to accept, and you want to make this decision based on which job is likely to make you happy. Being a very systematic person, you come up with three features associated with the offers, which are important for your happiness: whether the colleagues are supportive, work-hour flexibility, and whether the company is a start-up or not. 

In [151]:
offer_data = {
    # Features
    "supportive_colleagues" : [1, 0, 0],
    "work_hour_flexibility" : [0, 0, 1],
    "start_up" : [0, 1, 1],
    # Target
    "target": ["?", "?", "?"],
}

offer_df = pd.DataFrame(offer_data)
display(offer_df)    

,supportive_colleagues,work_hour_flexibility,start_up,target
0,1,0,0,?
1,0,0,1,?
2,0,1,1,?


Next, you ask the following questions to some of your friends (who you think have similar notions of happiness) regarding their jobs:

1. Do you have supportive colleagues? (1 for 'yes' and 0 for 'no')
2. Do you have flexible work hours? (1 for 'yes' and 0 for 'no')
3. Do you work for a start-up? (1 for 'start up' and 0 for 'non start up')
4. Are you happy in your job? (happy or unhappy)

You get the following data from this toy survey. Your goal is to train a machine learning model using this toy data and then use this model to predict which job is likely to make you happy. 

In [152]:
happiness_data = {
    # Features
    "supportive_colleagues" : [1, 1, 1, 0, 0, 1, 1, 0, 1, 0],
    "work_hour_flexibility" : [1, 1, 0, 1, 1, 0, 1, 0, 0, 0],
    "start_up" : [1, 0, 1, 0, 1, 0, 0, 1, 1, 0],
    # Target
    "target" : [
        "happy",
        "happy",
        "happy",
        "unhappy",
        "unhappy",
        "happy",
        "happy",
        "unhappy",
        "unhappy",
        "unhappy",
    ],
}

train_df = pd.DataFrame(happiness_data)
display(train_df)

,supportive_colleagues,work_hour_flexibility,start_up,target
0,1,1,1,happy
1,1,1,0,happy
2,1,0,1,happy
3,0,1,0,unhappy
4,0,1,1,unhappy
5,1,0,0,happy
6,1,1,0,happy
7,0,0,1,unhappy
8,1,0,1,unhappy
9,0,0,0,unhappy


**Question 2.1** <br> {points: 2}  

With this toy dataset, build a decision stump (decision tree with only 1 split) by hand by splitting on the condition `supportive_colleagues` == 1. 

What training accuracy would you get with this decision stump?

Save the accuracy as a fraction in an object named `supportive_colleagues_acc`. 

In [153]:
X = train_df[["supportive_colleagues"]]
y = train_df["target"]

In [154]:
happiness_model = DecisionTreeClassifier(max_depth = 1, random_state = 1)

In [155]:
happiness_model.fit(X, y)
happiness_predicted = happiness_model.predict(X)

display(happiness_predicted)

array(['happy', 'happy', 'happy', 'unhappy', 'unhappy', 'happy', 'happy',
       'unhappy', 'happy', 'unhappy'], dtype=object)

In [156]:
comparison_df = train_df[[
    'supportive_colleagues', 
    'target'
]]

comparison_df = comparison_df.assign(predicted = happiness_predicted)

display(comparison_df)

,supportive_colleagues,target,predicted
0,1,happy,happy
1,1,happy,happy
2,1,happy,happy
3,0,unhappy,unhappy
4,0,unhappy,unhappy
5,1,happy,happy
6,1,happy,happy
7,0,unhappy,unhappy
8,1,unhappy,happy
9,0,unhappy,unhappy


In [157]:
supportive_colleagues_acc = happiness_model.score(X, y)

display(supportive_colleagues_acc)

0.9

In [158]:
# check that the variable exists
assert 'supportive_colleagues_acc' in globals(
), "Please make sure that your solution is named 'supportive_colleagues_acc'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

**Question 2.2** 
<br> {points: 1}

The idea of a machine learning algorithm is to *fit* the best model on the given training data, which is in the form of feature vectors (`X`) and their corresponding targets(`y`),  and then using this model to *predict* targets for new examples (represented with feature vectors).

From `train_df`, create the feature table and save it in an object named `X` and the target in an object named `y`. 

In [159]:
X = train_df.drop(columns = ['target'])
display(X)

,supportive_colleagues,work_hour_flexibility,start_up
0,1,1,1
1,1,1,0
2,1,0,1
3,0,1,0
4,0,1,1
5,1,0,0
6,1,1,0
7,0,0,1
8,1,0,1
9,0,0,0


In [160]:
y = train_df['target']
display(y)

0      happy
1      happy
2      happy
3    unhappy
4    unhappy
5      happy
6      happy
7    unhappy
8    unhappy
9    unhappy
Name: target, dtype: object

In [161]:
t.test_2_2(X,y)

'Success'

**Question 2.3** 
<br> {points: 1}

Build a decision tree named `toy_tree` and fit it on the toy data using `sklearn`'s [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).


In [162]:
toy_tree = DecisionTreeClassifier(random_state = 1)
toy_tree.fit(X, y)

DecisionTreeClassifier(random_state=1)

In [163]:
t.test_2_3(toy_tree)

'Success'

**Question 2.4** <br> {points: 1}

Visualize the trained decision tree using the function `display_tree` that we have imported from the `display_tree` library already. 
Save it in an object named `toy_displayed`.

*Hint: use `?display_tree` to get more information about the function.*

In [164]:
?display_tree

Signature: display_tree(feature_names, tree)
Docstring:
Will create a decision tree visualization showing the features used on each split. 

Parameters
----------
feature_names : list
    The list of the features used in the decision tree
tree : sklearn.tree._classes.DecisionTreeClassifier
    The decision tree model we are using for prediction
    
Returns
-------
graphviz.files.Source 
    The decision tree visualization 
    
Examples
--------
>>> display_tree(X.columns, model])
File:      c:\my_files\school\school_work\engineering_bachelors\2022\keycapabilities_datascience\intro-ml-students\release\assignment2\display_tree.py
Type:      function


In [165]:
toy_displayed = display_tree(X.columns.to_list(), toy_tree)

In [166]:
# t.test_2_4(toy_displayed)

**Question 2.5** <br> {points: 1}

Score the decision tree on the training data (`X` and `y`).
Save the results in an object named `toy_score`. 

In [167]:
toy_score = toy_tree.score(X, y) 

display(toy_score)

0.9

In [168]:
t.test_2_5(toy_score)

'Success'

**Question 2.6** <br> {points: 1}

Predict on `X`. Add the results as a column named `predicted` in the `train_df` and name this new dataframe `predicted_train`. 

In [169]:
predicted_train = train_df.copy()
predicted_train = predicted_train.assign(predicted = toy_tree.predict(X))

display(predicted_train)

,supportive_colleagues,work_hour_flexibility,start_up,target,predicted
0,1,1,1,happy,happy
1,1,1,0,happy,happy
2,1,0,1,happy,happy
3,0,1,0,unhappy,unhappy
4,0,1,1,unhappy,unhappy
5,1,0,0,happy,happy
6,1,1,0,happy,happy
7,0,0,1,unhappy,unhappy
8,1,0,1,unhappy,happy
9,0,0,0,unhappy,unhappy


In [170]:
t.test_2_6(predicted_train)

'Success'

**Question 2.7** <br> {points: 1}

Do you get perfect training accuracy? 


A) Yes, the model correctly predicts every single observation

B) No, the model made a mistake likely because the decision tree wasn't complex enough.

C) No, there are two examples in the dataset with exactly the same feature values but different targets so the model makes a mistake on one of them. 

D No, the model is randomly predicting and therefore it won't get every single example correct. 

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer2_7`.*

In [171]:
feature_values = predicted_train.query('target != predicted').drop(['target', 'predicted'], axis = 1).to_dict(orient = 'records')[0]

# Find all examples with the same feature values as the incorrect prediction.
effective_df = predicted_train.copy()
for feature, value in feature_values.items() :
    effective_df = effective_df.query(f'{feature} == {value}')
display(effective_df)

,supportive_colleagues,work_hour_flexibility,start_up,target,predicted
2,1,0,1,happy,happy
8,1,0,1,unhappy,happy


In [172]:
answer2_7 = 'C'

In [173]:
t.test_2_7(answer2_7)

'Success'

**Question 2.8** <br> {points: 1}

Create a feature table from the `offer_df` (We don't know the target value in this case). 

*Save this in an object named `offer_X`*.

In [174]:
offer_X = offer_df.drop(columns = ['target'])

display(offer_X)

,supportive_colleagues,work_hour_flexibility,start_up
0,1,0,0
1,0,0,1
2,0,1,1


In [175]:
t.test_2_8(offer_X)

'Success'

**Question 2.9** <br> {points: 1}

Use the model `toy_tree` to predict which jobs from the `offer_df`, you will be happy working.  In other words, `predict` on `offer_X`.

Add a column to the `offer_df` dataframe named `predicted` and save the whole dataframe in an object named `pred_offer_df`.

In [176]:
pred_offer_df = offer_df.copy()
pred_offer_df = pred_offer_df.assign(predicted = toy_tree.predict(offer_X))
display(pred_offer_df)

,supportive_colleagues,work_hour_flexibility,start_up,target,predicted
0,1,0,0,?,happy
1,0,0,1,?,unhappy
2,0,1,1,?,unhappy


In [177]:
t.test_2_9(pred_offer_df)

'Success'

# 3. Exploratory Data Analysis and Decision Trees 

For the rest of the lab you'll be using a modified version of Kaggle's [Pokemon](https://www.kaggle.com/mlomuscio/pokemon?select=PokemonData.csv) dataset.
The dataset contains a number of features of pokemon's strength and weaknesses:

- `num`: ID for each Pokémon.
- `name`: Name of each Pokémon.
- `type`: Each Pokémon has a type, this determines weakness/resistance to attacks.
- `hp`: Hit points, or health, defines how much damage a Pokémon can withstand before fainting.
- `attack`: The base modifier for normal attacks (eg. Scratch, Punch).
- `defense`: The base damage resistance against normal attacks.
- `sp_atk`: Special attack, the base modifier for special attacks (e.g. fire blast, bubble beam).
- `sp_def`: The base damage resistance against special attacks.
- `total`: Sum of the `attack`, `defense`, `sp_atk`, and `sp_def` columns
- `speed`: Determines which Pokémon attacks first each round.
- `generation`: Number of generation.
- `legendary`: 1 if Legendary Pokémon, 0 if not.

In this question, our target is the `legendary` column.

In [178]:
pokemon = pd.read_csv('data/pokemon.csv')
display(pokemon.head())

,num,name,hp,attack,defense,sp_atk,sp_def,speed,total,generation,legendary,type
0,1,Bulbasaur,45,49,49,65,65,45,228,1,0,Grass
1,2,Ivysaur,60,62,63,80,80,60,285,1,0,Grass
2,3,Venusaur,80,82,83,100,100,80,365,1,0,Grass
3,3,VenusaurMega Venusaur,80,100,123,122,120,80,465,1,0,Grass
4,4,Charmander,39,52,43,60,50,65,205,1,0,Fire


**Question 3.1** <br> {points: 1}

Show information of each feature using `pd.DataFrame.info` on `pokemon` and answer the question below.

Select all that apply : 


A) There are `13` columns in the dataset.

B) The legendary column is of Dtype `int64`.

C) `5` columns have null values. 

D The name column is of Dtype `string`.

*Answer in the cell below using the uppercase letter associated with your answer. Place your answer(s) between `""` in a list, assign the correct answer to an object called `answer3_1`. For example ["A', "B"] is a possible answer*

In [179]:
display(pokemon.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   num         800 non-null    int64 
 1   name        800 non-null    object
 2   hp          800 non-null    int64 
 3   attack      800 non-null    int64 
 4   defense     800 non-null    int64 
 5   sp_atk      800 non-null    int64 
 6   sp_def      800 non-null    int64 
 7   speed       800 non-null    int64 
 8   total       800 non-null    int64 
 9   generation  800 non-null    int64 
 10  legendary   800 non-null    int64 
 11  type        800 non-null    object
dtypes: int64(10), object(2)
memory usage: 75.1+ KB


None

In [180]:
# This is wrong. The answer should be :
# answer3_1 = ['B']
answer3_1 = ['B', 'D']

In [181]:
t.test_3_1(answer3_1)

'Success'

**Question 3.2** <br> {points: 1}

Show summary statistics of each feature using `pd.DataFrame.describe` on `pokemon` and store it into a variable called `pokemon_summary`. 

In [182]:
pokemon_summary = pokemon.describe()

display(pokemon_summary)

,num,hp,attack,defense,sp_atk,sp_def,speed,total,generation,legendary
count,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000,800.00000
mean,362.813750,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,297.566250,3.32375,0.08125
std,208.343798,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,91.336677,1.66129,0.27339
min,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,90.000000,1.00000,0.00000
25%,184.750000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,225.000000,2.00000,0.00000
50%,364.500000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,295.000000,3.00000,0.00000
75%,539.250000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,365.000000,5.00000,0.00000
max,721.000000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,580.000000,6.00000,1.00000


In [183]:
t.test_3_2(pokemon_summary)

'Success'

**Question 3.3** <br> {points: 1}

Using the Altair skills that you learned in [**Programming in Python for Data Science**](https://prog-learn.mds.ubc.ca/en/), Take the code below that we started for you (between the `'''`) and copy it into the solution cell. Fill in the blank areas (`....`) so that the code produces histograms for the following features (in order) that show the distribution of the feature values, separated for 0 and 1 target values. 

- `hp`
- `attack`
- `defense`
- `sp_atk`
- `sp_def`
- `speed`
- `total`


```
def plot_histogram(df,feature):
    """
    plots a histogram of a decision trees feature
     
    Parameters
    ----------
    feature: str
        the feature name
    Returns
    -------
    altair.vegalite.v3.api.Chart
        an Altair histogram 
    """
    histogram = alt.Chart(df).mark_bar(
        opacity=0.7).encode(
        alt.X(feature, bin=alt.Bin(maxbins=50)),
        alt.Y('count()', stack=None),
        alt.Color(....)).properties(
        title= str.title(feature))
    return ....

feature_list = ....
figure_dict = dict()
for feature in .... :
    figure_dict.update({feature:plot_histogram(....,feature)})
figure_panel = alt.vconcat(*figure_dict.values())
figure_panel
```

In [184]:
display(pokemon)

,num,name,hp,attack,defense,sp_atk,sp_def,speed,total,generation,legendary,type
0,1,Bulbasaur,45,49,49,65,65,45,228,1,0,Grass
1,2,Ivysaur,60,62,63,80,80,60,285,1,0,Grass
2,3,Venusaur,80,82,83,100,100,80,365,1,0,Grass
3,3,VenusaurMega Venusaur,80,100,123,122,120,80,465,1,0,Grass
4,4,Charmander,39,52,43,60,50,65,205,1,0,Fire
...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,50,100,150,100,150,50,500,6,1,Rock
796,719,DiancieMega Diancie,50,160,110,160,110,110,540,6,1,Rock
797,720,HoopaHoopa Confined,80,110,60,150,130,70,450,6,1,Psychic
798,720,HoopaHoopa Unbound,80,160,60,170,130,80,520,6,1,Psychic


In [185]:
def plot_histogram(df, feature, target = 'legendary', maxbins = 50):

    """
    plots a histogram of a decision trees feature
     
    Parameters
    ----------
    feature: str
        the feature name
    target: str, optional
        the target name
    maxbins: int, optional
        the maximum number of data bins on the x-axis
    
    Returns
    -------
    altair.vegalite.v3.api.Chart
        an Altair histogram 
    """

    histogram = alt.Chart(df).mark_bar(
        opacity = 0.7).encode(
        x = alt.X(feature, bin = alt.Bin(maxbins = 50)),
        y = alt.Y('count()', stack = None),
        color = alt.Color(
            'legendary:N', scale = alt.Scale(
                domain = [0, 1], 
                range = ['gray', 'green']
            ), legend = alt.Legend(title = 'isLegendary')
        )
    ).properties(
        title = str.title(feature)
    )

    return histogram

In [186]:
feature_list = [
    'hp',
    'attack',
    'defense',
    'sp_atk',
    'sp_def',
    'speed',
    'total',
]

display(feature_list)

['hp', 'attack', 'defense', 'sp_atk', 'sp_def', 'speed', 'total']

In [187]:
figure_dict = dict()
for feature in feature_list :
    figure_dict.update({feature : plot_histogram(pokemon, feature, target = 'legendary', maxbins = 50)})
figure_panel = alt.vconcat(*figure_dict.values())
display(figure_panel)

c:\Users\Muntakim\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [188]:
t.test_3_3(plot_histogram,figure_panel,figure_dict)

'Success'

***Side Exploration***

In [189]:
display(pokemon.query('legendary == 1'))

,num,name,hp,attack,defense,sp_atk,sp_def,speed,total,generation,legendary,type
156,144,Articuno,90,85,100,95,125,85,405,1,1,Ice
157,145,Zapdos,90,90,85,125,90,100,390,1,1,Electric
158,146,Moltres,90,100,90,125,85,90,400,1,1,Fire
162,150,Mewtwo,106,110,90,154,90,130,444,1,1,Psychic
163,150,MewtwoMega Mewtwo X,106,190,100,154,100,130,544,1,1,Psychic
...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,50,100,150,100,150,50,500,6,1,Rock
796,719,DiancieMega Diancie,50,160,110,160,110,110,540,6,1,Rock
797,720,HoopaHoopa Confined,80,110,60,150,130,70,450,6,1,Psychic
798,720,HoopaHoopa Unbound,80,160,60,170,130,80,520,6,1,Psychic


In [190]:
chart_df = pokemon.copy() 
chart_df['legendary'] = chart_df['legendary'].astype('bool')
chart_df = chart_df.groupby('legendary').count().reset_index()
display(chart_df)

,legendary,num,name,hp,attack,defense,sp_atk,sp_def,speed,total,generation,type
0,False,735,735,735,735,735,735,735,735,735,735,735
1,True,65,65,65,65,65,65,65,65,65,65,65


In [191]:
legendary_chart = alt.Chart(chart_df).mark_arc().encode(
    theta = alt.Theta("num:Q", stack = True), 
    color = alt.Color(
        'legendary:N', scale = alt.Scale(
            domain = [False, True], 
            range = ['gray', 'green']
        ), legend = alt.Legend(title = 'isLegendary')
    )
).properties(
    title = "Legendary Statuses"
)

display(legendary_chart)

c:\Users\Muntakim\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

**Question 3.4** 
<br> {points: 2}

Which feature appears to be the most useful in differentiating the target classes?


*Answer in the cell below by putting the feature name between `""` and assign it to an object called `answer3_4`.*

In [192]:
answer3_4 = 'total'

In [193]:
# check that the variable exists
assert 'answer3_4' in globals(
), "Please make sure that your solution is named 'answer3_4'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

**Question 3.5** 
<br> {points: 1}

Suppose for a particular feature, the histograms of that feature are identical for the two target classes. Does that mean the feature is not useful for predicting the target class?

A) If the histograms are identical then there is no way differentiate each target value and so the feature is not useful. 

B) If the histograms are identical then we only need to use that feature for predicting the target value.

C) If the histograms are identical, the feature might still be useful because it may be predictive in conjunction with other features.

D) If the histograms are identical, the feature might still be useful but only with other models.  


*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_5`.*


In [194]:
answer3_5 = 'C'

In [195]:
t.test_3_5(answer3_5)

'Success'

**Question 3.6** 
<br> {points: 1}

Note that the dataset includes a categorical features labeled `type`. Do you think this feature could be useful in predicting whether the pokemon was legendary or not and would there be any difficulty in using it in our decision tree?

A) Yes, it would be useful but adding categorical features into a model needs special attention.

B) Yes, it would be useful and we shouldn't have any difficulty adding them into our model. 

C) No, We have enough features to predict with, the added `type` column would not add anything significant.

D) No, and categorical features would need special attention to add to our model.


*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer3_6`.*


In [196]:
answer3_6 = 'A'

In [197]:
t.test_3_6(answer3_6)

'Success'

# 4. Hyperparameters

**Question 4.1** 
<br> {points: 1}

Create your $X$ and $y$ objects so that you X dataframe contains the columns: 
- `hp`
- `attack`
- `defense`
- `sp_atk`
- `sp_def`
- `speed`
- `total`
- `generation`

and your $y$ dataframe is the `legendary` column.

*Save each in the respective object names `X` and `y`* 

In [198]:
X = pokemon[['hp', 'attack', 'defense', 'sp_atk', 'sp_def', 'speed', 'total', 'generation']]
y = pokemon['legendary']

In [199]:
t.test_4_1(X,y)

'Success'

**Question 4.2** 
<br> {points: 3}

In this exercise, you'll explore the `max_depth` hyperparameter within the range 1 to 15. See the [`DecisionTreeClassifier` documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) for more details.

To do so, you will need to make a for loop for each value between 1-15 that: 
- Creates a model named `pokemon_tree`.
- Sets the `max_depth` hyperparameter to the value it's iterating on. 
- Sets the argument `random_state=8`. 
- Fits each model on X and y.
- Appends the model's score to the list depth_accuracy..



In [200]:
depth_accuracy = [DecisionTreeClassifier(random_state = 8, max_depth = i).fit(X, y).score(X, y) for i in range(1, 15 + 1)]
display(depth_accuracy)

[0.91875,
 0.95,
 0.95,
 0.96125,
 0.98,
 0.98875,
 0.99125,
 0.995,
 0.995,
 0.995,
 0.99625,
 0.9975,
 0.99875,
 0.99875,
 0.99875]

In [201]:
t.test_4_2(depth_accuracy)

'Success'

**Question 4.3** <br>
{points: 3} <br> 

Make a dataframe that contains the tree depth and scores and name it `depth_scores_df`

It should look something like this:

|    | max_depth      |   accuracy |
|---:|:--------------|-----------:|
|  0 | 1    |       # |
|  1 | 2 |       # |
|  2 | 3       |       # |
| ...| ...|...|
| 14 |15|#|


In [202]:
depth_scores_df = pd.DataFrame({'max_depth' : range(1, 15 + 1), 'accuracy' : depth_accuracy})
display(depth_scores_df)

,max_depth,accuracy
0,1,0.91875
1,2,0.95000
2,3,0.95000
3,4,0.96125
4,5,0.98000
5,6,0.98875
6,7,0.99125
7,8,0.99500
8,9,0.99500
9,10,0.99500


In [203]:
t.test_4_3(depth_scores_df)

'Success'

**Question 4.4** <br>
 {points: 1} <br>
 
Using altair, make a `mark_line()` plot which displays the depth of the decision tree on the *x*-axis and the `depth_accuracy` on the *y*-axis.
Make sure it has the dimensions `width=500, height=300`. Don't forget to give it a title and the plot `depth_acc_plot`


In [204]:
?alt.Scale

Init signature:
alt.Scale(
    align=Undefined,
    base=Undefined,
    bins=Undefined,
    clamp=Undefined,
    constant=Undefined,
    domain=Undefined,
    domainMax=Undefined,
    domainMid=Undefined,
    domainMin=Undefined,
    exponent=Undefined,
    interpolate=Undefined,
    nice=Undefined,
    padding=Undefined,
    paddingInner=Undefined,
    paddingOuter=Undefined,
    range=Undefined,
    rangeMax=Undefined,
    rangeMin=Undefined,
    reverse=Undefined,
    round=Undefined,
    scheme=Undefined,
    type=Undefined,
    zero=Undefined,
    **kwds,
)
Docstring:     
Scale schema wrapper

Mapping(required=[])

Attributes
----------

align : anyOf(float, :class:`ExprRef`)
    The alignment of the steps within the scale range.

    This value must lie in the range ``[0,1]``. A value of ``0.5`` indicates that the
    steps should be centered within the range. A value of ``0`` or ``1`` may be used to
    shift the bands to one side, say to position them adjacent to an axis.

   

In [205]:
depth_acc_plot = alt.Chart(depth_scores_df).mark_line().encode(
    x = alt.X('max_depth:Q', title = 'Max Depth'),
    y = alt.Y('accuracy:Q', title = 'Depth Accuracy', scale = alt.Scale(zero = False))
).properties(
    width = 500,
    height = 300,
    title = 'Accuracy by Max Depth'
)

display(depth_acc_plot)

c:\Users\Muntakim\AppData\Local\Programs\Python\Python310\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [206]:
t.test_4_4(depth_acc_plot)

'Success'

# 5 Decision Tree Regressor

Let's use the real estate data set that we saw in Assignment 1 and see if we can improve our $R^2$ from last time. 

For this question we are using a dataset obtained from [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set) that contains the market historical data of real estate valuation collected from Sindian District, New Taipei City in Taiwan.

The columns in the dataset can be explained as follows: 

- `date`: the transaction date (for example, 2013.250=2013 March, 2013.500=2013 June, etc.)
- `house_age`: the house age (unit: year)
- `distance_station`: the distance to the nearest Mass Rapid Transit (MRT) station (unit: meter)
- `num_stores`: the number of convenience stores in the living circle on foot (integer)(a ***living circle*** is a residential space with similar local characteristics, and daily behaviors)
- `latitude`: the geographic coordinate, latitude. (unit: degree)
- `longitude`: the geographic coordinate, longitude. (unit: degree)
- `price`: house price per unit area (10000 New Taiwan Dollar/Ping,where Ping is a local unit of area, 1 Ping = 3.3 meter squared)


In [207]:
housing_df = pd.read_csv('data/real_estate.csv')
display(housing_df.head())

,house_age,distance_station,num_stores,latitude,longitude,price
0,32.0,84.87882,10,24.98298,121.54024,37.9
1,19.5,306.59470,9,24.98034,121.53951,42.2
2,13.3,561.98450,5,24.98746,121.54391,47.3
3,13.3,561.98450,5,24.98746,121.54391,54.8
4,5.0,390.56840,5,24.97937,121.54245,43.1


**Question 5.1** 
<br> {points: 1}

Create your $X$ and $y$ objects.   
For the `X` dataframe make sure that you are not including `price`. Since our `y` (target) is the `price` column.        

*Save each in the respective object names `X` and `y`* 

In [208]:
X = housing_df.drop(columns = ['price'])
y = housing_df['price']

In [209]:
t.test_5_1(X,y)

'Success'

**Question 5.2** 
<br> {points: 1}

Build a Decision tree Regressor named `tree_reg`. Make sure to import DecisionTreeRegressor from  the sklearn.tree library. 
Train it on the variables `X` and `y` that we made in question 5.1. Save the score in a variable named `tree_score`. 

In [210]:
from sklearn.tree import DecisionTreeRegressor

In [211]:
tree_reg = DecisionTreeRegressor(random_state = 1)
tree_reg.fit(X, y)
tree_score = tree_reg.score(X, y)

display(tree_score)

0.9893300488110535

In [212]:
t.test_5_2(tree_score)

'Success'

**Question 5.3** 
<br> {points: 2}

Does the model do better than the Dummy Regressor we used in assignment 1? 

A) Both models Dummy Regressor and Decision Tree Regressor do about the same.

B) Dummy Regressor does moderately better.

C) Decision Tree Regressor does moderately better.

D) Dummy Regressor does much better than the Decision Tree Regressor.

E) Decision Tree Regressor does much better than the Dummy Regressor.



*Answer in the cell below using the uppercase letter associated with your answer. Place your answer between `""`, assign the correct answer to an object called `answer5_3`.*


In [213]:
answer5_3 = 'E'

In [214]:
# check that the variable exists
assert 'answer5_3' in globals(
), "Please make sure that your solution is named 'answer5_3'"

# This test has been intentionally hidden. It will be up to you to decide if your solution
# is sufficiently good.

## Before Submitting 

Before submitting your assignment please do the following:

- Read through your solutions
- **Restart your kernel and clear output and rerun your cells from top to bottom** 
- Makes sure that none of your code is broken 
- Verify that the tests from the questions you answered have obtained the output "Success"

This is a simple way to make sure that you are submitting all the variables needed to mark the assignment. This method should help avoid losing marks due to changes in your environment.  

## Attributions
- Fertitily Diagnosis Dataset: - [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Fertility)

*David Gil, Jose Luis Girela, Joaquin De Juan, M. Jose Gomez-Torres, and
Magnus Johnsson. Predicting seminal quality with artificial intelligence
methods. Expert Systems with Applications, 39(16):12564 â€“ 12573, 2012*

- Real Estate Dataset - [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set)

*Yeh, I. C., & Hsu, T. K. (2018). Building real estate valuation models with comparative approach through case-based reasoning. Applied Soft Computing, 65, 260-271.*


- MDS DSCI 571 - Supervised Learning I - [MDS's GitHub website](https://github.com/UBC-MDS/DSCI_571_sup-learn-1) 
